In [13]:
import numpy as np
from mpgm.mpgm.sample_generation.graph_generators import HubGraphGenerator

In [61]:
def generate_lpgm_samples(lambda_true:float, lambda_noise:float, adj_matrix:np.ndarray, 
                         nr_samples:int) -> np.ndarray:
    A = adj_matrix
    n = nr_samples
    p = A.shape[0]
    
    B = create_B(A)
    
    C = lambda_true * A
    upper_tri = C[np.triu_indices(p, 1)]
    nonzero_tri = upper_tri[upper_tri != 0]
    Y_lambdas = np.append(np.repeat(lambda_true, p), nonzero_tri)
    Y = np.zeros((len(Y_lambdas), n))
    for ii, ylam in enumerate(Y_lambdas):
        Y[ii, :] = np.random.poisson(ylam, (1, n))
    
    X = np.matmul(B, Y) # X is p x n
    X = np.transpose(X) # X is now n x p
    
    # Add noise.
    E = np.random.poisson(lambda_noise, (n, p))
    X = X + E
    return X

def create_B(adj_matrix:np.ndarray) -> np.ndarray:
    A = adj_matrix
    nrows = A.shape[0]
    ncols = A.shape[1]
    if nrows != ncols:
        raise Exception("create_B: Adjacency matrix is not square.")
    
    B = np.identity(nrows)
    for ii in range(nrows):
        for jj in range(ii+1, ncols):
            if A[ii][jj] == 1:
                new_col = np.zeros((nrows, 1))
                new_col[ii] = 1
                new_col[jj] = 1
                B = np.hstack((B, new_col))
    
    return B

In [60]:
np.random.seed(1)
gg = HubGraphGenerator(nr_hubs=1)
A = gg.generate_graph(nr_variables=4)
print(generate_lpgm_samples(1, 0.5, A, 20))

[[ 1.  5.  3.  8.]
 [ 2.  7.  5.  6.]
 [ 0.  2.  2.  2.]
 [ 0.  7.  1.  6.]
 [ 1.  2.  4.  4.]
 [ 2.  0.  4.  5.]
 [ 3.  1.  4.  5.]
 [ 3.  3.  4. 10.]
 [ 3.  2.  5.  5.]
 [ 1.  2.  2.  1.]
 [ 3.  2.  5.  2.]
 [ 2.  5.  3.  1.]
 [ 1.  2.  4.  4.]
 [ 2.  6.  3.  6.]
 [ 4.  3.  3.  5.]
 [ 2.  3.  8.  6.]
 [ 2.  2.  0.  5.]
 [ 2.  0.  4.  3.]
 [ 7.  2.  2.  8.]
 [ 1.  2.  3.  6.]]


In [54]:
A.T

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.]])

In [55]:
np.transpose(A)

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 1., 1., 0.]])